
We want to classify yoyo in to 4 flavors using features such as package color, sweetness level.

Supposed we want to use 1 VS All algorithm. To do this.


1) How many classifiers we need to train?

In [ ]:
#given n class we need log2(n) class 

2) How do we train each classifier? (Which one to call + and which one to call -)

3) Write a table for perfect answer for each class.

4) If the answer from all the classifier for a sample looks like 
$$(0.1,0.2,0.3,0.6)$$
What class should it be?

what about
$$(0.6,0.6,0.7,0.7)$$

5) How many answers we can get wrong before the classifier start getting confused?

6) How can we do better against wrong answer?

7) Write down the exhaustive matrix for 4 classes.

# Real Exercise

Read glass.data and build a classifier to classify glass.

What is this glass?

$$1.5,14.8,0.00,2.00,73.00,0.00,8.00,1.7,0.00$$

In [161]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from scipy.optimize import minimize

In [162]:
df = pd.read_csv('glass.data.txt', header=None)
df[10][df[10] > 4] = df[10][df[10] > 4] -1 #shift > 4 back by 1 
df[10] = df[10] - 1 #make it start from 1 
df[0] = 1 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [87]:
code_table = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ]) #row -> class, column -> classifier 

In [95]:
class Logistic:
    def __init__(self, xs, ys):
        self.xs = xs
        self.ys = ys   #ys must be 1 and -1 
        self.w = self.train()
        
    def theta(self, s):
            return 1/(1+np.exp(-np.clip(s,-50,50)))
    
    def train(self):
        def cost(w):
            return -np.sum(np.log(self.theta(self.ys*np.dot(self.xs,w))))
        res = minimize(cost, [0]*self.xs.shape[1])
        return res.x
    
    def classify(self,xs):
        return self.theta(np.dot(xs, self.w))
    

In [189]:
class MulticlassClassifier:
    def __init__(self, xs, ys, codetable): #ys = 0,1,2,3,4,...
        self.xs = xs 
        self.ys = ys
        self.codetable = codetable
        self.classifiers = self.train()   

    def train(self):
        n = self.codetable.shape[1]
        classifiers = [self.train_ith_classifier(i) for i in range(n)]
        return classifiers
    
    def train_ith_classifier(self,i):
        new_ys = []
        for y in self.ys:
            new_ys.append(self.codetable[y][i]*2-1)
        log = Logistic(self.xs, new_ys)
        return log 
        
    def classify(self,xs): # x -> 0,1,2,3,4
        prob = np.array([cl.classify(xs) for cl in self.classifiers])
        print(prob)
        distance = [np.linalg.norm(prob-cl)**2 for cl in self.codetable]
        print(distance)
        return np.argmin(distance)
    

In [190]:
xs = df[df.columns[:-1]].values
ys = df[10]
multi = MulticlassClassifier(xs, ys, code_table)

In [191]:
multi.classify([1.0,1.5,14.8,0.00,2.00,73.00,0.00,8.00,1.7,0.00])

[1.41180144e-05 3.16827543e-02 9.99999735e-01 7.48243518e-01
 1.92874985e-22 4.60655297e-19]
[2.560843392537356, 2.49750611988734, 0.5608721593895462, 1.0643845925146325, 2.5608716285662294, 2.5608716285662294]


2

In [215]:
from pprint import pprint
def exhaustive(n):
    ncol = 2**(n-1)
    table = np.zeros((n, ncol))
    for i in range(n):
        nzero = int(ncol/2**(i))
        if i != 0:
            print(i, int(ncol/nzero/2))
            row = ([0]*nzero + [1]*nzero)*int(ncol/nzero/2)   #basically, it's 2**i-1
        else:
            row = ([0]*nzero)
        table[i] = row
    return table[:,:-1]
        
        
print(exhaustive(4))

1 1
2 2
3 4
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 0. 0. 1.]
 [0. 1. 0. 1. 0. 1. 0.]]
